In [1]:
# importing relevant module
import csv
import re
from collections import defaultdict



In [2]:
# function to read a file located at the given file_path with a pipe-separated values format
def content(file_path):
    with open(file_path, 'r') as file:
        return [block for block in csv.reader(file, delimiter='|')][1:]


In [3]:
# function to process basketball-related data, and organize it into a structured format that includes team names and player statistics.
def assemble(variation):
    outcome = {"home_team": {"name": "", "players_data": []}, "away_team": {"name": "", "players_data": []}}

    # Extract home and away team names from the first play
    away_team_name, home_team_name = variation[0][3], variation[0][4]

    # Initialize participant details lists for home and away teams
    home_team = outcome["home_team"]["players_data"]
    away_team = outcome["away_team"]["players_data"]

    for trend in variation:
        quality_home_team, important_group, current_plan = trend[4], trend[2], trend[7]

        # Use regex to match the participant name in the trend description
        good_fit = re.match(r"^([A-Z]\. [A-Z]\w{1,}|[A-Z]\. [A-Z]\w{1,}\-\w{1,})", current_plan)

        if good_fit:
            player_name = good_fit.group(1)

            participant = {
                        "player_name": player_name,
                        "FG": 0, "3P": 0, "3PM": 0, "3PA": 0,
                        "2P": 0, "2PM": 0,
                        "FT": 0, "FTM": 0, "FTA": 0,
                        "ORB": 0, "DRB": 0, "TRB": 0,
                        "AST": 0, "STL": 0, "BLK": 0,
                        "TOV": 0, "PF": 0, "PTS": 0,
                        "MCPFT": 0, "MICPFT": 0
                    }

            # Determine the desired team's participant details list
            desired_group = home_team if important_group == home_team_name else away_team

            # Add participant to the desired team's players_data if not already present
            if participant not in desired_group:
                desired_group.append(participant)

    # Set the extracted team names
    outcome["home_team"]["name"] = home_team_name
    outcome["away_team"]["name"] = away_team_name

    return outcome



In [4]:
# extract patterns and relevant information to match events and update player statistics accordingly
def directory():
    patterns = [
        "makes 3-pt", "misses 3-pt", "makes 2-pt", "misses 2-pt",
        "Offensive rebound by", "Defensive rebound by",
        "block by", "steal by", "Turnover by", "assist by",
        "Personal foul by",
        "makes free throw", "misses free throw",
        "makes clear path free throw", "misses clear path free throw"
    ]

    actions = ["3P", "3PM", "2P", "2PM", "ORB", "DRB", "BLK", "STL", "TOV", "AST", "PF", "FT", "FTM", "MCPFT", "MICPFT"]

    return [(f"([A-Z]\. [A-Z]\w{{1,}}|[A-Z]\. [A-Z]\w{{1,}}-\\w{{1,}}) {pattern}", action) for pattern, action in zip(patterns, actions)]


In [5]:
# function to process events stored in the instant list, find matches with a specified pattern in each event's description, and update participant counts
def transfer(participants, pattern, instant, occasion):
    # Create a defaultdict to store the participant names and their corresponding counts
    player_counts = defaultdict(int)

    # Iterate through the instant and update the participant counts
    for trend in instant:
        match = re.compile(pattern).search(trend[7])
        if match:
            player_name = match.group(1)
            player_counts[player_name] += 1

    # Update the participants dictionary with the incremented counts
    for team in ["home_team", "away_team"]:
        for participant in participants[team]["players_data"]:
            player_name = participant["player_name"]
            participant[occasion] += player_counts[player_name]



In [6]:
# function to analyze NBA (National Basketball Association) game data.
def analyse_nba_game():
    # Load variation and initialize details dictionary
    variation = content("/content/drive/MyDrive/Colab Notebooks/Data/nba_game_warriors_thunder_20181016.txt")
    details = assemble(variation)

    # Get upload patterns and apply to participant details
    catalog = directory()
    for control, model in catalog:
        transfer(details, control, variation, model)

    # Calculate derived participant statistics
    for team in details.values():
        for participant in team["players_data"]:
            participant["FG"], participant["FGA"], participant["3PA"] = participant["3P"] + participant["2P"], participant["3P"] + participant["2P"], participant["3P"] + participant["3PM"]
            participant["FT"], participant["FTA"] = participant["FT"] * 1 + participant["MCPFT"] + participant["MICPFT"], participant["FT"] + participant["FTM"]
            participant["PTS"], participant["TRB"] = participant["3P"] * 3 + participant["2P"] * 2 + participant["FT"], participant["DRB"] + participant["ORB"]

            # Calculate percentages only if the denominator is not zero
            if participant["FGA"] != 0:
                participant["FG%"] = round(participant["FG"] / participant["FGA"], 3)
            if participant["3PA"] != 0:
                participant["3P%"] = round(participant["3P"] / participant["3PA"], 3)
            if participant["FTA"] != 0:
                participant["FT%"] = round(participant["FT"] / participant["FTA"], 3)

    # Remove unnecessary holders from participant details
    rid_holder = ["FGM", "2P", "2PM", "3PM", "FTM", "MCPFT", "MICPFT"]
    for team in details.values():
        for participant in team["players_data"]:
            for holder in rid_holder:
                if holder in participant:
                    del participant[holder]

    return details


In [7]:
def evaluate_sum(group_details):
    # Define participant statistics to calculate
    participant_reports = ["FG", "FGA", "3P", "3PA", "FT", "FTA", "ORB", "DRB", "AST", "STL", "BLK", "TOV", "PF", "PTS"]

    # Calculate sum for each participant statistic
    tally = {report: sum(participant[report] for participant in group_details["players_data"]) for report in participant_reports}

    # Initialize percentages fields
    tally["FG%"] = 0
    tally["3P%"] = 0
    tally["FT%"] = 0

    # Calculate percentages only if the denominator is not zero
    if tally["FGA"] != 0:
        tally["FG%"] = round(tally["FG"] / tally["FGA"], 3)
    if tally["3PA"] != 0:
        tally["3P%"] = round(tally["3P"] / tally["3PA"], 3)
    if tally["FTA"] != 0:
        tally["FT%"] = round(tally["FT"] / tally["FTA"], 3)

    # Calculate total rebounds
    tally["TRB"] = tally["ORB"] + tally["DRB"]

    # Return the calculated statistics
    return tally["FG"], tally["FGA"], tally["FG%"], tally["3P"], tally["3PA"], tally["3P%"], tally["FT"], tally["FTA"], tally["FT%"], tally["ORB"], tally["DRB"], tally["TRB"], tally["AST"], tally["STL"], tally["BLK"], tally["TOV"], tally["PF"], tally["PTS"]



In [8]:
def main_counter():
    # Get participant details from the analysis
    players_data = analyse_nba_game()

    def print_data(participant):
        # Print participant statistics with formatted columns
        print(f'{participant["player_name"]:<20} | {participant.get("FG", 0):<5} | {participant.get("FGA", 0):<5} | {participant.get("FG%", 0):<6} | {participant.get("3P", 0):<5} | {participant.get("3PA", 0):<5} | {participant.get("3P%", 0):<6} | {participant.get("FT", 0):<5} | {participant.get("FTA", 0):<5} | {participant.get("FT%", 0):<6} | {participant.get("ORB", 0):<5} | {participant.get("DRB", 0):<5} | {participant.get("TRB", 0):<5} | {participant.get("AST", 0):<5} | {participant.get("STL", 0):<5} | {participant.get("BLK", 0):<5} | {participant.get("TOV", 0):<5} | {participant.get("PF", 0):<5} | {participant.get("PTS", 0):<5} ')

    # Print header row
    print(f'{"Players name":<20} | {"FG":<5} | {"FGA":<5} | {"FG%":<6} | {"3P":<5} | {"3PA":<5} | {"3P%":<6} | {"FT":<5} | {"FTA":<5} | {"FT%":<6} | {"ORB":<5} | {"DRB":<5} | {"TRB":<5} | {"AST":<5} | {"STL":<5} | {"BLK":<5} | {"TOV":<5} | {"PF":<5} | {"PTS":<5} ')

    # Print participant details and team totals
    for team, group_details in players_data.items():
        for participant in group_details["players_data"]:
            print_data(participant)

        # Calculate team totals
        fg, fga, fgp, p3, p3a, p3p, ft, fta, ftp, orb, drb, trb, ast, stl, blk, tov, pf, pts = evaluate_sum(group_details)

        # Print team totals
        print(f'{"Team Totals":<20} | {fg:<5} | {fga:<5} | {fgp:<6} | {p3:<5} | {p3a:<5} | {p3p:<6} | {ft:<5} | {fta:<5} | {ftp:<6} | {orb:<5} | {drb:<5} | {trb:<5} | {ast:<5} | {stl:<5} | {blk:<5} | {tov:<5} | {pf:<5} | {pts:<5} ')
        print('\n')
    print("\n")

# Call the main counter function to display the analysis results
main_counter()


Players name         | FG    | FGA   | FG%    | 3P    | 3PA   | 3P%    | FT    | FTA   | FT%    | ORB   | DRB   | TRB   | AST   | STL   | BLK   | TOV   | PF    | PTS   
S. Curry             | 11    | 11    | 1.0    | 5     | 9     | 0.556  | 5     | 3     | 1.667  | 0     | 0     | 0     | 0     | 0     | 0     | 0     | 0     | 32    
K. Durant            | 9     | 9     | 1.0    | 0     | 5     | 0.0    | 9     | 10    | 0.9    | 0     | 0     | 0     | 0     | 0     | 0     | 0     | 0     | 27    
D. Jones             | 6     | 6     | 1.0    | 0     | 0     | 0      | 0     | 0     | 0      | 0     | 0     | 0     | 0     | 0     | 0     | 0     | 0     | 12    
D. Green             | 1     | 1     | 1.0    | 0     | 1     | 0.0    | 0     | 0     | 0      | 0     | 0     | 0     | 0     | 0     | 0     | 0     | 0     | 2     
K. Thompson          | 5     | 5     | 1.0    | 1     | 8     | 0.125  | 3     | 3     | 1.0    | 0     | 0     | 0     | 0     | 0     | 0     | 0     | 0